<a href="https://colab.research.google.com/github/chuviethongfpt/atom-assignments/blob/main/Home_Assignment__5_CVH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os
import numpy as np

# TODO 1: BIGQUERY & SQL

In [2]:
!ls

env_variable.json  healthy-cubist-315301-d974eee70c6a.json  sample_data


In [3]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [4]:
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN']

In [5]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

In [6]:
user_dict = {'user_id':[],'name':[],'display_name':[],'real_name':[],'title':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

user_df = pd.DataFrame(user_dict) 
user_df = user_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
user_df.head()

,user_id,name,display_name,real_name,title,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,NaN,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,NaN,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,NaN,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,NaN,False


In [7]:
endpoint2 = "https://slack.com/api/conversations.list"
data = {'types': 'public_channel,private_channel'} # -> CHECK: API Docs https://api.slack.com/methods/conversations.list/test
response_json = requests.post(endpoint2, headers=headers, data=data).json() 
channel_dat = response_json['channels']

channel_dict = {'channel_id':[], 'channel_name':[], 'is_channel':[],'creator':[],'created_at':[],'topics':[],'purpose':[],'num_members':[]}
for i in range(len(channel_dat)):
  channel_dict['channel_id'].append(channel_dat[i]['id'])
  channel_dict['channel_name'].append(channel_dat[i]['name'])
  channel_dict['is_channel'].append(channel_dat[i]['is_channel'])
  channel_dict['creator'].append(channel_dat[i]['creator'])
  channel_dict['created_at'].append(dt.fromtimestamp(float(channel_dat[i]['created'])))
  channel_dict['topics'].append(channel_dat[i]['topic']['value'])
  channel_dict['purpose'].append(channel_dat[i]['purpose']['value'])
  channel_dict['num_members'].append(channel_dat[i]['num_members'])

channel_df = pd.DataFrame(channel_dict) 
channel_df = channel_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
channel_df.head()

,channel_id,channel_name,is_channel,creator,created_at,topics,purpose,num_members
0,C01B4PVGLVB,general,True,U01BE2PR6LU,2020-09-23 10:25:03,NaN,This is the one channel that will always inclu...,64
1,C01BYH7JHB5,contents,True,U01BE2PR6LU,2020-10-05 07:28:09,NaN,NaN,12
2,C01CAMNCJJV,branding-design,True,U01AVDY7JET,2020-10-05 03:16:30,NaN,NaN,9
3,C01U6P7LZ8F,atom-assignment1,True,U01BE2PR6LU,2021-04-17 07:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44
4,C01UL6K1C7L,atom-week1,True,U01BE2PR6LU,2021-04-16 17:44:32,TOPIC: Data Strategy. Metrics Design. Spreadsh...,TOPIC: Data Strategy. Metrics Design. Spreadsh...,43


In [8]:
endpoint3 = "https://slack.com/api/conversations.history"

In [9]:
msg_dict = {'channel_id':[],'msg_id':[], 'msg_ts':[], 'user_id':[], 'latest_reply':[],'reply_user_count':[],'reply_users':[],'github_link':[]}
for channel_id, channel_name in zip(channel_df['channel_id'], channel_df['channel_name']):
  print('Channel ID: {} - Channel Name: {}'.format(channel_id, channel_name))
  try:
    data = {"channel": channel_id} 
    response_json = requests.post(endpoint3, data=data, headers=headers).json()
    msg_ls = response_json['messages']
    for i in range(len(msg_ls)):
      if 'client_msg_id' in msg_ls[i].keys():
        msg_dict['channel_id'].append(channel_id)
        msg_dict['msg_id'].append(msg_ls[i]['client_msg_id'])
        msg_dict['msg_ts'].append(dt.fromtimestamp(float(msg_ls[i]['ts'])))
        msg_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'] if 'latest_reply' in msg_ls[i].keys() else 0))) ## -> No reply: 1970-01-01
        msg_dict['user_id'].append(msg_ls[i]['user'])
        msg_dict['reply_user_count'].append(msg_ls[i]['reply_users_count'] if 'reply_users_count' in msg_ls[i].keys() else 0)
        msg_dict['reply_users'].append(msg_ls[i]['reply_users'] if 'reply_users' in msg_ls[i].keys() else 0) 
        ## -> Censor message contains tokens
        text = msg_ls[i]['text']
        github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text)
        msg_dict['github_link'].append(github_link[0] if len(github_link) > 0 else np.nan)
  except:
    print('====> '+ str(response_json))
        

msg_df = pd.DataFrame(msg_dict)
msg_df = msg_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
msg_df.head()

Channel ID: C01B4PVGLVB - Channel Name: general
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01BYH7JHB5 - Channel Name: contents
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01CAMNCJJV - Channel Name: branding-design
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01U6P7LZ8F - Channel Name: atom-assignment1
Channel ID: C01UL6K1C7L - Channel Name: atom-week1
Channel ID: C01ULCHGN75 - Channel Name: atom-general
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C020VMT58JK - Channel Name: topics-data-analytics
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C0213MNH9L6 - Channel Name: topics-python
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C0213N56M2A - Channel Name: topics-materials
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C021FSDN7LJ - Channel Name: atom-assignment2
Channel ID: C021KLB0DSB - Channel Name: discuss-group3
Channel ID: C021KLB90GP - Channel Name: discuss-group4
Channel ID: C0220

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link
0,C01U6P7LZ8F,005AD620-BD06-4123-BC42-6118DAA73BA4,2021-05-13 14:09:48.011000,U01UTGRMGEQ,2021-05-13 18:01:34.021400,4,"[U01V00P4S2V, U01UTGRMGEQ, U01UMJ451V1, U01V00...",NaN
1,C01U6P7LZ8F,D86A48D2-D953-4504-92BF-D9240B2549A8,2021-05-10 04:51:46.004700,U01UJ9LG5U5,2021-05-15 09:14:31.003600,2,"[U01USGKQ771, U01V00KHHHP]",https://github.com/danhpcv/
2,C01U6P7LZ8F,246fb20c-8739-4664-9bcf-14f57aee17f7,2021-05-10 04:35:39.003800,U01UMC08KL2,2021-05-18 08:09:59.000800,2,"[U01UTGS5ZNY, U01UMC08KL2]",https://github.com/auslynnguyen/
3,C01U6P7LZ8F,4214088f-ccdc-4e8e-9c19-677ae3f439a1,2021-05-10 03:07:35.001200,U01UTGRQS9J,2021-05-15 09:28:16.004000,2,"[U01VB632FPS, U01V00KHHHP]",NaN
4,C01U6P7LZ8F,cfad12f7-bf9c-4209-b563-f495f378fb1f,2021-05-08 06:43:25.006900,U01BE2PR6LU,1970-01-01 00:00:00.000000,0,0,NaN


In [24]:
dtc_groups = pd.read_csv('datacracy_groups.csv')
dtc_groups.head()

,name,DataCracy_role
0,slackbot,BTC
1,loclexuan26392,BTC
2,locle.ds,BTC
3,maianhdang.ftu,BTC
4,huyenhoang.design,BTC


In [17]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'healthy-cubist-315301-d974eee70c6a.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [18]:
sheet_index_no = 0
spreadsheet_key = '18sB_gZMyz0hEKCaY1_Arv1-_EFwT675kT3kDBm7PVuw' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

In [14]:
# ACCESS GOOGLE SHEET
sheet_index_no = 1
spreadsheet_key = '18sB_gZMyz0hEKCaY1_Arv1-_EFwT675kT3kDBm7PVuw' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.add_worksheet(title= 'channel_df', rows= "100", cols= "20") #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet,  channel_df) #-> Upload assignment_df vào Sheet 2 trong Spreadsheet
print('Done!')

Done!


In [15]:
# ACCESS GOOGLE SHEET
sheet_index_no = 2
spreadsheet_key = '18sB_gZMyz0hEKCaY1_Arv1-_EFwT675kT3kDBm7PVuw' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.add_worksheet(title= 'msg_df', rows= "100", cols= "20") #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet,  msg_df) #-> Upload assignment_df vào Sheet 2 trong Spreadsheet
print('Done!')

Done!


In [26]:
sheet_index_no = 3
spreadsheet_key = '18sB_gZMyz0hEKCaY1_Arv1-_EFwT675kT3kDBm7PVuw' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.add_worksheet(title= 'dtc_groups', rows= "100", cols= "20") #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, dtc_groups) #-> Upload assignment_df vào Sheet 2 trong Spreadsheet
print('Done!')

Done!


#TODO 2: Python Pandas
Kaggle là cộng đồng lớn nhất thế giới về Data Science với các cuộc thi/thử thách ML, các công cụ và tài liệu để học và phát triển kỹ năng

* Tạo account của bạn trên Kaggle
* Coi tutorial và làm 6 sets bài tập trên Kaggle HERE
* Bạn có thể làm bài trực tiếp trên Kaggle (cũng là dạng notebook online như Colab). Toàn bộ notebook sẽ lưu trên phần Code trong Profile Account của bạn

In [ ]:
# Copy các link Kaggle sau:
## 1. Link tới Kaggle Account của bạn -----> 
## 2. Link tới các bài tập
## Pandas 1: ---> https://www.kaggle.com/chuviethong/exercise-creating-reading-and-writing/edit
## Pandas 2: ---> https://www.kaggle.com/chuviethong/exercise-indexing-selecting-assigning/edit
## Pandas 3: ---> https://www.kaggle.com/chuviethong/exercise-summary-functions-and-maps/edit
## Pandas 4: ---> https://www.kaggle.com/chuviethong/exercise-grouping-and-sorting/edit
## Pandas 5: ---> https://www.kaggle.com/chuviethong/exercise-data-types-and-missing-values/edit
## Pandas 6: ---> https://www.kaggle.com/chuviethong/exercise-renaming-and-combining/edit

# TODO 3: Python Pandas

In [31]:
msg_cha = pd.merge(msg_df, channel_df, how='left', on='channel_id')
user_dtc = pd.merge(user_df, dtc_groups, how='left', on='name')
merge_df = pd.merge(msg_cha, user_dtc, how='left', on='user_id')

In [36]:
df = merge_df.loc[(merge_df.github_link.notnull()) & (merge_df['channel_name'].str.startswith('atom-assignment')) & (merge_df['DataCracy_role'].str.startswith('Learner'))]
df.head(3)

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link,channel_name,is_channel,creator,created_at,topics,purpose,num_members,name,display_name,real_name,title,is_bot,DataCracy_role
1,C01U6P7LZ8F,D86A48D2-D953-4504-92BF-D9240B2549A8,2021-05-10 04:51:46.004700,U01UJ9LG5U5,2021-05-15 09:14:31.003600,2,"[U01USGKQ771, U01V00KHHHP]",https://github.com/danhpcv/,atom-assignment1,True,U01BE2PR6LU,2021-04-17 07:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44,danhpcv,Danh Phan,Danh Phan,NaN,False,Learner_Gr3
2,C01U6P7LZ8F,246fb20c-8739-4664-9bcf-14f57aee17f7,2021-05-10 04:35:39.003800,U01UMC08KL2,2021-05-18 08:09:59.000800,2,"[U01UTGS5ZNY, U01UMC08KL2]",https://github.com/auslynnguyen/,atom-assignment1,True,U01BE2PR6LU,2021-04-17 07:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44,auslyn.nguyen,NaN,Minh-Thu Nguyen Hoang,NaN,False,Learner_Gr4
8,C01U6P7LZ8F,4abf215d-ff30-44c0-9384-9da735a374c1,2021-05-07 16:29:48.000700,U01USGKQ771,2021-05-11 02:52:00.000200,1,[U01UMC0N5U2],https://github.com/TrangNgo1709/,atom-assignment1,True,U01BE2PR6LU,2021-04-17 07:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44,trang.ngo1709,Trang,Ngo Thi Minh Trang,NaN,False,Learner_Gr1


In [37]:
df1=df.groupby('channel_name').apply(lambda df: df.loc[df.msg_ts.idxmin()])
df1[['user_id','display_name','DataCracy_role', 'msg_ts','github_link' ]]

,user_id,display_name,DataCracy_role,msg_ts,github_link
channel_name,,,,,
atom-assignment1,U01UMJ5AQ0K,Dương Phan Nam Hưng,Learner_Gr1,2021-04-21 06:00:34.000800,https://github.com/duongphannamhung/
atom-assignment2,U01UQGZHSTZ,Toan Tran,Learner_Gr2,2021-05-12 15:50:48.004000,https://github.com/toantran2206/
atom-assignment3,U01UTGRMGEQ,Nguyen Hong Hanh,Learner_Gr4,2021-05-16 12:08:08.018900,https://github.com/nguyenhonghanh2511/
atom-assignment4,U01UJ9M3P1T,Cao Nguyen Huy Hoang,Learner_Gr2,2021-05-25 17:46:52.015200,https://github.com/cnhhoang850/
atom-assignment5,U01UTGT8CVA,Nguyễn Thảo Nguyên,Learner_Gr3,2021-06-02 10:34:22.003500,https://github.com/ngttnguyen/


In [38]:
df2=df.groupby('channel_name').apply(lambda df: df.loc[df.msg_ts.idxmax()])
df2[['user_id','display_name','DataCracy_role', 'msg_ts','github_link' ]]

,user_id,display_name,DataCracy_role,msg_ts,github_link
channel_name,,,,,
atom-assignment1,U01UJ9LG5U5,Danh Phan,Learner_Gr3,2021-05-10 04:51:46.004700,https://github.com/danhpcv/
atom-assignment2,U01UTGRMGEQ,Nguyen Hong Hanh,Learner_Gr4,2021-05-22 05:39:34.015500,https://github.com/nguyenhonghanh2511/
atom-assignment3,U01USGKQ771,Trang,Learner_Gr1,2021-05-27 15:30:16.003200,https://github.com/TrangNgo1709/
atom-assignment4,U01U6JQB695,NaN,Learner_Gr1,2021-06-04 03:12:24.000800,https://github.com/Tenderriver/
atom-assignment5,U01UJ9P1S3X,Thao Le,Learner_Gr3,2021-06-03 16:46:34.009600,https://github.com/lethuthao1368/
